In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# import the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
from IPython.display import display

In [ ]:
# read the dataset
telecom = pd.read_csv('telecom_churn_data.csv')
telecom

In [ ]:
# let's check info
telecom.info(verbose=1)

It is a big dataset, there are 99999 rows, and 226 columns mostly includes integer datatype either in int or float format

In [ ]:
# lets describe the dataset
telecom.describe(include = 'all')

### Data Preparation

#### Filter High- Value Customers

In [ ]:
# high-value customers are those who recharge with an amount more than value "X" i.e., 70 percentile of average recharge amount in first two months. 
# here we will caluculate the total amount spend by the customers on recharge date.
# we will multiply two columns to get the total amount 'av_rech_amt_data_6' and 'total_rech_data_6' for 6,7,8,9 months


telecom['total_rech_data_amt_6'] = telecom['av_rech_amt_data_6'] * telecom['total_rech_data_6']
telecom['total_rech_data_amt_7'] = telecom['av_rech_amt_data_7'] * telecom['total_rech_data_7']
telecom['total_rech_data_amt_8'] = telecom['av_rech_amt_data_8'] * telecom['total_rech_data_8']
telecom['total_rech_data_amt_9'] = telecom['av_rech_amt_data_9'] * telecom['total_rech_data_9']

the columns in the dataset are very high, we need to drop variables which are not useful for furthur analysis, columns with high missing values, and impute number of columns of same category and delete the originals etc.,

In [ ]:
# now we dont need the columns "av_rech_amt_data_x,total_rech_data_x" x =[6,7,8,9], lets drop them

telecom.drop(['total_rech_data_6','total_rech_data_7','total_rech_data_8','total_rech_data_9',
'av_rech_amt_data_6','av_rech_amt_data_7','av_rech_amt_data_8','av_rech_amt_data_9'],axis = 1,inplace = True)

In [ ]:
# lets find out the average recharge done in the first two months(june & july) - the good phase
# total amount spend would be the sum of total data recharge done & total call/sms recharges

telecom_av_rech_6_7 = (telecom['total_rech_amt_6'].fillna(0) + telecom['total_rech_amt_7'].fillna(0) + telecom['total_rech_data_amt_6'].fillna(0) 
+ telecom['total_rech_data_amt_7'].fillna(0))/2

In [ ]:
# extract percentile_70 of the calculated average amount

percentile_70_6_7 = np.percentile(telecom_av_rech_6_7, 70.0)
print("70 percentile is : ", percentile_70_6_7)

In [ ]:
#filtering the dataset to 70 percentile

telecom_cust = telecom[telecom_av_rech_6_7 >= percentile_70_6_7]
telecom_cust.shape

In [ ]:
# new column 'churn' is introduced in telecom_cust dataset, 1=churn and 0=non-churn
# lets calculate churn or non-churn for '_9' columns

telecom_cust['churn'] = np.where(telecom_cust[['total_ic_mou_9','total_og_mou_9','vol_2g_mb_9','vol_3g_mb_9']].sum(axis=1)==0,1,0)
telecom_cust

In [ ]:
# lets find out churn rate for the 'telecom_cust' dataset
100*(telecom_cust['churn'].value_counts()/len(telecom_cust))

from the above observation 91% are non-churn customers and 8% are churn customers

In [ ]:
# now we will check the columns which have no difference in values, many values have single value like 1
# since such columns are not useful after checking lets drop them

for i in telecom_cust.columns:
    if telecom_cust[i].nunique() == 1:
        print(i)
        telecom_cust.drop(i,axis=1,inplace=True)

In [ ]:
# lets check the shape of dataset after dropping single value variables
telecom_cust.shape

In [ ]:
# Now lets check the null values present in the dataset

null_perc = 100*(telecom_cust.isnull().sum()/len(telecom_cust)).sort_values(ascending=False)
null_perc.head(20)

In [ ]:
null_perc=null_perc[null_perc.values>=30]

col_list=list(null_perc.index)
telecom_cust.drop(columns=col_list,axis=1,inplace=True)
telecom_cust.shape

In [ ]:
# lets check for columns that are object datatype

object_col_data = telecom_cust.select_dtypes(include=['object'])
print(object_col_data.iloc[0])

In [ ]:
# from the above data we got the columns which contains dates
# lets convert them to dates column to date_time datatype


for col in object_col_data.columns:
    telecom_cust[col] = pd.to_datetime(telecom_cust[col])

telecom_cust.shape

As we know the for model being stable enough it is important the variance should be low 
Now lets check the data, variables with high correlation values and drop them

In [ ]:
cor = telecom_cust.corr()

cor.loc[:,:] = np.tril(cor, k=-1)

cor = cor.stack()
pd.options.display.max_rows = None
cor[(cor > 0.60) | (cor < -0.60)].sort_values()

In [ ]:
# we will drop the columns with high correlation (+/- 60%)
drop_col_list = ['std_og_t2t_mou_6','std_og_t2t_mou_7','std_og_t2t_mou_8','std_og_t2t_mou_9',
                'std_og_t2m_mou_6','std_og_t2m_mou_7','std_og_t2m_mou_8','std_og_t2m_mou_9',
                'total_og_mou_6','total_og_mou_7','total_og_mou_8',
                'loc_ic_t2t_mou_6','loc_ic_t2t_mou_7','loc_ic_t2t_mou_8','loc_ic_t2t_mou_9',
                'loc_ic_t2m_mou_6','loc_ic_t2m_mou_7','loc_ic_t2m_mou_8','loc_ic_t2m_mou_9',
                'std_ic_t2m_mou_6','std_ic_t2m_mou_7','std_ic_t2m_mou_8','std_ic_t2m_mou_9',
                'total_ic_mou_6','total_ic_mou_7','total_ic_mou_8',
                'total_rech_amt_6','total_rech_amt_7','total_rech_amt_8','total_rech_amt_9',
                'vol_3g_mb_6','vol_3g_mb_7','vol_3g_mb_8',
                'loc_og_t2t_mou_9',
                'loc_og_t2f_mou_9',
                'loc_og_t2m_mou_9',
                 'loc_ic_t2f_mou_6','loc_ic_t2f_mou_7','loc_ic_t2f_mou_8','loc_ic_t2f_mou_9',
                'date_of_last_rech_6','date_of_last_rech_7','date_of_last_rech_8']

In [ ]:
telecom_cust.drop(drop_col_list, axis=1, inplace=True)
telecom_cust.shape

In [ ]:
# Now we will delete 9th month columns because we would predict churn/non-churn later based on data from the 1st 3 months
cols_to_drop = [col for col in telecom_cust.columns if '_9' in col]
print(cols_to_drop)

telecom_cust.drop(cols_to_drop, axis=1, inplace=True)

telecom_cust.shape

In [ ]:
# lets check the dataset again
(telecom_cust.isnull().sum() * 100 / len(telecom_cust)).sort_values(ascending = False)

- We have dropped columns more than 30% missing values
- we have dropped columns with '_9' tag ending
- we changed the data type of 'object' type columns


- we are left with columns of null values with less than just 3.91%, now we have deal with those columns, since the columns are high in number
- we are looking through the variables which are useful and which are not useful, and dropping the not useful columns.

In [ ]:
drop_cols_2 = ['onnet_mou_6','onnet_mou_7','onnet_mou_8','offnet_mou_6','offnet_mou_7','offnet_mou_8',
               'roam_ic_mou_6','roam_ic_mou_7','roam_ic_mou_8','roam_og_mou_6','roam_og_mou_7','roam_og_mou_8','loc_og_mou_6',
               'loc_og_t2t_mou_6','loc_og_t2t_mou_7','loc_og_t2t_mou_8','loc_og_t2m_mou_6','loc_og_t2m_mou_7','loc_og_t2m_mou_8',
               'loc_og_t2f_mou_6','loc_og_t2f_mou_7','loc_og_t2f_mou_8','loc_og_t2c_mou_6','loc_og_t2c_mou_7','loc_og_t2c_mou_8',
               'loc_og_mou_7','loc_og_mou_8','std_og_t2f_mou_6','std_og_t2f_mou_7','std_og_t2f_mou_8',
               'std_og_mou_6','std_og_mou_7','std_og_mou_8','isd_og_mou_6','isd_og_mou_7','isd_og_mou_8',
               'spl_og_mou_6','spl_og_mou_7','spl_og_mou_8','og_others_6','og_others_7','og_others_8',
               'loc_ic_mou_6','loc_ic_mou_7','loc_ic_mou_8','std_ic_t2t_mou_6','std_ic_t2t_mou_7',
              'std_ic_t2t_mou_8','std_ic_t2f_mou_6','std_ic_t2f_mou_7','std_ic_t2f_mou_8',
               'std_ic_mou_6','std_ic_mou_7','std_ic_mou_8','spl_ic_mou_6','spl_ic_mou_7','spl_ic_mou_8',
               'isd_ic_mou_6','isd_ic_mou_7','isd_ic_mou_8','ic_others_6','ic_others_7','ic_others_8']

In [ ]:
telecom_cust.drop(drop_cols_2, axis=1, inplace=True)
telecom_cust.shape

In [ ]:
((telecom_cust.isnull().sum())/(telecom_cust.shape[0]))*100

from the above data it is clear that, the left columns have no null values, so let's move forward to check the skewness and outliers of the data.

since all the left over columns are of integer datatypes so no need to do univariate categorical analysis,

### Derive some new features from existing columns


In [ ]:
# create a new colulmn, which would be average  of 6th & 7th months
# lets first create list of columns belonging to 6th and 7th months

col_list = telecom_cust.filter(regex='_6|_7').columns.str[:-2]
col_list.unique()

print (telecom_cust.shape)

# lets take the average now
for idx, col in enumerate(col_list.unique()):
    avg_col_name = "avg_"+col+"_av67" # lets create the column name dynamically
    col_6 = col+"_6"
    col_7 = col+"_7"
    telecom_cust[avg_col_name] = (telecom_cust[col_6]  + telecom_cust[col_7])/ 2

In [ ]:
# we dont need columns from which we have derived new features, we will drop those columns

print ("dimension of the updated dataset after creating dervied features:",telecom_cust.shape)
col_to_drop = telecom_cust.filter(regex='_6|_7').columns
telecom_cust.drop(col_to_drop, axis=1, inplace=True)

print("dimension of the dataset after dropping un-necessary columns:",telecom_cust.shape)

In [ ]:
# lets now conevrt AON in months
telecom_cust['aon_mon'] = telecom_cust['aon']/30
telecom_cust.drop('aon', axis=1, inplace=True)
telecom_cust['aon_mon'].head()

#### Uni-variate Numerical

#### Churn Vs Other important features

In [ ]:
# assign all the numerical columns to new df
num_cols = telecom_cust.columns

In [ ]:
# plot the distplot to check the distribution of the data and the skewness
plt.figure(figsize=[20,20])
i=1
for col in num_cols:
    plt.subplot(5,7,i)
    sns.distplot(telecom_cust[col])
    plt.tight_layout()
    i = i+1

In [ ]:
# plot between tenure and revenue
telecom_cust[['aon_mon', 'arpu_8']].plot.scatter(x = 'aon_mon',y='arpu_8')

In [ ]:
# for better understanding, lets plot the boxplot

plt.figure(figsize=[20,20])
i=1
for col in num_cols:
    plt.subplot(5,7,i)
    sns.boxplot(y=telecom_cust[col],x=telecom_cust["churn"])
    plt.tight_layout()
    i=i+1

- though, the variables have outliers. the data in the columns are important to analyse and outliers treatment can lead to loss of important values.
- So, no outlier treatment is done
#### the data is skewed dealing with such data could harm the results. and Logistic Regression is heavily influenced by the outliers. so just skipped treating the same

In [ ]:
i=1
plt.figure(figsize=[20,20])
for col in num_cols:
    plt.subplot(5,7,i)
    sns.boxplot(y=telecom_cust[col])
    plt.title(col)
    i=i+1

In [ ]:
plt.figure(figsize=([30,40]))
sns.heatmap(telecom_cust.corr(),annot=True,cmap='RdYlGn')

In [ ]:
### Checking the Churn Rate
churn_rate = (sum(telecom_cust['churn'])/len(telecom_cust['churn'].index))*100
churn_rate

## Model Preparation

In [ ]:
telecom_cust.info()

since there are no categorical columns left in the dataset we are not creating dummy variables

In [ ]:
# first copy the dataframe to new dataframe
# then drop the mobile_number column which is not important for building the model
new_telecom = telecom_cust.copy()
new_telecom.drop('mobile_number', axis=1, inplace=True)
new_telecom.head()

In [ ]:
# lets create X & y dataset for model building
#X willnot have "churn" and y will only have "churn"

X = new_telecom.drop(['churn'], axis=1)
y = new_telecom['churn']

new_telecom.drop('churn', axis=1, inplace=True)

In [ ]:
# split the dateset into train and test datasets
# for that import the libraries required

from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, train_size=0.7, random_state=1)

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
# import libraries for model building

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

import statsmodels.api as sm

from sklearn.feature_selection import RFE

In [ ]:
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)

As we know this dataset is highly imbalanced and skewed, because churn rate is very low and non-churn rate is very high in such scenario we have deal with balancing the dataset.

Synthetic Minority Oversampling Technique-SMOTE is used

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='auto')
X_tr,y_tr = smote.fit_resample(X_train,y_train)

I tried doing svm technique, but unfortunately either my laptop is slow or not capable of handling this huge dataset so im directly diving to 'logistic regression'

#### Checking the Correlation Matrix

In [ ]:
plt.figure(figsize = (20,10))
sns.heatmap(X_tr.corr(),annot = True)
plt.show()

#### Feature Selection Using RFE

In [ ]:
# Logistic regression model
logm1 = sm.GLM(y_train,(sm.add_constant(X_train)), family = sm.families.Binomial())
logm1.fit().summary()

In [ ]:
lr = LogisticRegression()

In [ ]:
# ger
rfe = RFE(lr, 15)   
rfe = rfe.fit(X_tr, y_tr)
rfe_features = list(new_telecom.columns[rfe.support_])
print(rfe_features)

In [ ]:
X_rfe = X_train.columns[rfe.support_]

In [ ]:
X_train.columns[~rfe.support_]

In [ ]:
X_train_sm = sm.add_constant(X_train[X_rfe])
logm2 = sm.GLM(y_train,X_train_sm, family = sm.families.Binomial())
res = logm2.fit()
res.summary()

In [ ]:
# Getting the predicted values on the train set
y_train_pred = res.predict(X_train_sm)
y_train_pred[:10]

In [ ]:
y_train_pred = y_train_pred.values.reshape(-1)
y_train_pred[:10]

In [ ]:
y_train_pred_final = pd.DataFrame({'Churn':y_train.values, 'Churn_Prob':y_train_pred})
y_train_pred_final['CustID'] = y_train.index
y_train_pred_final.head()

In [ ]:
y_train_pred_final['predicted'] = y_train_pred_final.Churn_Prob.map(lambda x: 1 if x > 0.5 else 0)

# Let's see the head
y_train_pred_final.head()

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
# Confusion matrix 
confusion = metrics.confusion_matrix(y_train_pred_final.Churn, y_train_pred_final.predicted )
print(confusion)

In [ ]:
# Let's check the overall accuracy.
print(metrics.accuracy_score(y_train_pred_final.Churn, y_train_pred_final.predicted))

#### Checking VIFs

In [ ]:
# Check for the VIF values of the feature variables. 
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs
vif = pd.DataFrame()
vif['Features'] = X_train[X_rfe].columns
vif['VIF'] = [variance_inflation_factor(X_train[X_rfe].values, i) for i in range(X_train[X_rfe].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
X_rfe = X_rfe.drop('avg_total_rech_num_av67',1)
X_rfe

In [ ]:
X_train_sm = sm.add_constant(X_train[X_rfe])
logm2 = sm.GLM(y_train,X_train_sm, family = sm.families.Binomial())
res = logm2.fit()
res.summary()

In [ ]:
y_train_pred = res.predict(X_train_sm).values.reshape(-1)

In [ ]:
y_train_pred[:10]

In [ ]:
y_train_pred_final['Churn_Prob'] = y_train_pred

In [ ]:
# Creating new column 'predicted' with 1 if Churn_Prob > 0.5 else 0
y_train_pred_final['predicted'] = y_train_pred_final.Churn_Prob.map(lambda x: 1 if x > 0.5 else 0)
y_train_pred_final.head()

In [ ]:
# Let's take a look at the confusion matrix again 
confusion = metrics.confusion_matrix(y_train_pred_final.Churn, y_train_pred_final.predicted )
confusion

In [ ]:
# Let's check the overall accuracy.
print(metrics.accuracy_score(y_train_pred_final.Churn, y_train_pred_final.predicted))

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X_train[X_rfe].columns
vif['VIF'] = [variance_inflation_factor(X_train[X_rfe].values, i) for i in range(X_train[X_rfe].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

### Decision Trees

In [ ]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt = DecisionTreeClassifier(max_depth=3,random_state = 42)
dt.fit(X_train, y_train)

In [ ]:
# Importing required packages for visualization
from IPython.display import Image  
from six import StringIO  
from sklearn.tree import export_graphviz
import pydotplus, graphviz

In [ ]:
# plotting tree with max_depth=3
dot_data = StringIO()  

export_graphviz(dt, out_file=dot_data, filled=True, rounded=True,
                feature_names=X.columns, 
                class_names=['No Disease', "Disease"])

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(graph.create_png())
#Image(graph.create_png(),width=800,height=900)
#graph.write_pdf("dt_heartdisease.pdf")

#### Evaluating model performance

In [ ]:
y_train_pred = dt.predict(X_train)
y_test_pred = dt.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print(accuracy_score(y_train, y_train_pred))
confusion_matrix(y_train, y_train_pred)

In [ ]:
print(accuracy_score(y_test, y_test_pred))
confusion_matrix(y_test, y_test_pred)

#### Creating helper functions to evaluate model performance and help plot the decision tree

In [ ]:
def get_dt_graph(dt_classifier):
    dot_data = StringIO()
    export_graphviz(dt_classifier, out_file=dot_data, filled=True,rounded=True,
                    feature_names=X.columns, 
                    class_names=['Disease', "No Disease"])
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
    return graph

In [ ]:
def evaluate_model(dt_classifier):
    
    print("Train Accuracy :", accuracy_score(y_train, dt_classifier.predict(X_train)))
    print("Train Confusion Matrix:")
    print(confusion_matrix(y_train, dt_classifier.predict(X_train)))
    print("-"*50)
    print("Test Accuracy :", accuracy_score(y_test, dt_classifier.predict(X_test)))
    print("Test Confusion Matrix:")
    print(confusion_matrix(y_test, dt_classifier.predict(X_test)))

In [ ]:
evaluate_model(dt)

In [ ]:
gph = get_dt_graph(dt)
Image(gph.create_png())

#### without setting any hyperparameters

In [ ]:
dt_default = DecisionTreeClassifier(random_state=42)
dt_default.fit(X_train, y_train)

In [ ]:
gph = get_dt_graph(dt_default)
Image(gph.create_png())

In [ ]:
evaluate_model(dt_default)

we have already plotted a decision tree based on controlled depth so skipping this condition and doing furthur conditions to check the accuracy of the model

##### Specifying minimum samples before split

In [ ]:
dt_min_split = DecisionTreeClassifier(min_samples_split=20)
dt_min_split.fit(X_train, y_train)

In [ ]:
gph = get_dt_graph(dt_min_split) 
Image(gph.create_png())

In [ ]:
evaluate_model(dt_min_split)

#### Specifying minimum samples in leaf node

In [ ]:
dt_min_leaf = DecisionTreeClassifier(min_samples_leaf=50, random_state=42)
dt_min_leaf.fit(X_train, y_train)

In [ ]:
gph = get_dt_graph(dt_min_leaf)
Image(gph.create_png())

In [ ]:
evaluate_model(dt_min_leaf)

##### Using Entropy instead of Gini

In [ ]:
dt_min_leaf_entropy = DecisionTreeClassifier(min_samples_leaf=20, random_state=42, criterion="entropy")
dt_min_leaf_entropy.fit(X_train, y_train)

In [ ]:
gph = get_dt_graph(dt_min_leaf_entropy)
Image(gph.create_png())

In [ ]:
evaluate_model(dt_min_leaf_entropy)

### Hyper parameter tuning

In [ ]:
dt = DecisionTreeClassifier(random_state=42)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Create the parameter grid based on the results of random search 
params = {
    'max_depth': [2, 3, 5, 10, 20],
    'min_samples_leaf': [5, 10, 20, 50, 100],
    'criterion': ["gini", "entropy"]
}

In [ ]:
# Instantiate the grid search model
grid_search = GridSearchCV(estimator=dt, 
                           param_grid=params, 
                           cv=4, n_jobs=-1, verbose=1, scoring = "accuracy")

In [ ]:
%%time
grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_score_

In [ ]:
grid_search.best_estimator_

In [ ]:
dt_best = grid_search.best_estimator_

In [ ]:
evaluate_model(dt_best)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, dt_best.predict(X_test)))

In [ ]:
gph = get_dt_graph(dt_best)
Image(gph.create_png())

In [ ]:
from sklearn.metrics import plot_roc_curve

In [ ]:
plot_roc_curve(dt_best, X_train, y_train)
plt.show()

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(n_estimators=10, max_depth=4, max_features=5, random_state=100, oob_score=True)

In [ ]:
%%time
rf.fit(X_train, y_train)

In [ ]:
rf.oob_score_

In [ ]:
plot_roc_curve(rf, X_train, y_train)
plt.show()

#### Hyper-parameter tuning for the Random Forest

In [ ]:
rf = RandomForestClassifier(random_state=42, n_jobs=-1)

In [ ]:
params = {
    'max_depth': [2,3,5,10,20],
    'min_samples_leaf': [5,10,20,50,100,200],
    'n_estimators': [10, 25, 50, 100]
}

In [ ]:
grid_search = GridSearchCV(estimator=rf,
                           param_grid=params,
                           cv = 4,
                           n_jobs=-1, verbose=1, scoring="accuracy")

In [ ]:
%%time
grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_score_

In [ ]:
rf_best = grid_search.best_estimator_
rf_best

In [ ]:
plot_roc_curve(rf_best, X_train, y_train)
plt.show()

In [ ]:
rf_best.feature_importances_

In [ ]:
imp_df = pd.DataFrame({
    "Varname": X_train.columns,
    "Imp": rf_best.feature_importances_
})

In [ ]:
imp_df.sort_values(by="Imp", ascending=False).head(10)